# 空间分布的传热

下一个创建可重用子系统的例子会有些变化。在这一节中，我们不仅会与本章前面例子里一样，展示如何创建可重用的子系统模型。而且子系统模型将使用部件实例组成的数组。该数组的大小可被用于控制结果的空间分辨率。

## 无层级系统

从如下所示无层级系统级模型开始：我们有3个热电容和5个热导体，代表的被等分为3段的杆、分段之间的热传导以及每个分段与环境的热传导。

<img src="./figures8/FlatRod.svg" width=1600 align=center>

模型由热容以及热容之间的热导体组成。这里有3个热容以及5个热导体。热量施加在该系统的左端。系统右端的温度传感器测量最右热容的温度变化。

模型的Modelica实现如下：

```Modelica
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model FlatRod "Modeling a heat transfer in a rod in a without subsystems"
  Modelica.Thermal.HeatTransfer.Sources.PrescribedHeatFlow heating
    "Heating actuator"
    annotation (Placement(transformation(extent={{-60,50},{-40,70}})));
  Modelica.Blocks.Sources.Step bc(height=10, startTime=0.5) "Heat profile"
    annotation (Placement(transformation(extent={{-90,50},{-70,70}})));
  Modelica.Thermal.HeatTransfer.Components.HeatCapacitor C1(C=0.1, T(fixed=true))
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        origin={-60,2})));
  Modelica.Thermal.HeatTransfer.Components.ThermalConductor G1(G=1.2)
    annotation (Placement(transformation(extent={{-40,-30},{-20,-10}})));
  Modelica.Thermal.HeatTransfer.Components.HeatCapacitor C2(C=0.1, T(fixed=true))
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        origin={0,2})));
  Modelica.Thermal.HeatTransfer.Components.ThermalConductor G2(G=1.2)
    annotation (Placement(transformation(extent={{20,-30},{40,-10}})));
  Modelica.Thermal.HeatTransfer.Components.HeatCapacitor C3(C=0.1, T(fixed=true))
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        origin={60,2})));
  Modelica.Thermal.HeatTransfer.Sensors.TemperatureSensor sensor
    annotation (Placement(transformation(extent={{80,-30},{100,-10}})));
  Modelica.Thermal.HeatTransfer.Components.ThermalConductor wall1(G=0.9)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={-60,-40})));
  Modelica.Thermal.HeatTransfer.Sources.FixedTemperature ambient(T=293.15)
    "Ambient temperature" annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={0,-80})));
  Modelica.Thermal.HeatTransfer.Components.ThermalConductor wall2(G=0.9)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={0,-40})));
  Modelica.Thermal.HeatTransfer.Components.ThermalConductor wall3(G=0.9)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={60,-40})));
equation
  connect(bc.y, heating.Q_flow) annotation (Line(
      points={{-69,60},{-60,60}},
      color={0,0,127}, smooth=Smooth.None));
  connect(C1.port, G1.port_a) annotation (Line(
      points={{-60,-8},{-60,-20},{-40,-20}},
      color={191,0,0}, smooth=Smooth.None));
  connect(C2.port, G2.port_a) annotation (Line(
      points={{0,-8},{0,-20},{20,-20}},
      color={191,0,0}, smooth=Smooth.None));
  connect(G2.port_b, C3.port) annotation (Line(
      points={{40,-20},{60,-20},{60,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(G1.port_b, C2.port) annotation (Line(
      points={{-20,-20},{0,-20},{0,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(heating.port, C1.port) annotation (Line(
      points={{-40,60},{-30,60},{-30,20},{-80,20},{-80,-20},{-60,-20},{-60,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(wall1.port_b, C1.port) annotation (Line(
      points={{-60,-30},{-60,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(wall1.port_a, ambient.port) annotation (Line(
      points={{-60,-50},{-60,-60},{0,-60},{0,-70}},
      color={191,0,0}, smooth=Smooth.None));
  connect(wall2.port_a, ambient.port) annotation (Line(
      points={{0,-50},{0,-50},{0,-70}},
      color={191,0,0}, smooth=Smooth.None));
  connect(wall3.port_a, ambient.port) annotation (Line(
      points={{60,-50},{60,-60},{0,-60},{0,-70}},
      color={191,0,0}, smooth=Smooth.None));
  connect(wall3.port_b, C3.port) annotation (Line(
      points={{60,-30},{60,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(sensor.port, C3.port) annotation (Line(
      points={{80,-20},{60,-20},{60,-8}},
      color={191,0,0}, smooth=Smooth.None));
  connect(C2.port, wall2.port_b) annotation (Line(
      points={{0,-8},{0,-19},{0,-30},{0,-30}},
      color={191,0,0},
      smooth=Smooth.None));
end FlatRod;
```
模拟这个系统，我们可以从下面的图内看到最右边热容的温度响应：

<img src="./figures8/FR.png" width=500 align=center>


## 分段杆子系统

从理论上讲，我们可以把杆分为$N$段。然后，这些分段间有$N−1$条热传导路径。而分段和环境之间则有$N$条热传导路径。前面例子中配置为$N=3$。但我们可以创建一个以$N$为参数的子系统模型。要做到这一点，我们不能简单地将热容拖放到模型中，然后将其与热导体连接起来。因为我们不知道系统确切等分为几份。

相反，我们将使用组件数组来代表这一系列热容和热导体。所得到的Rod模型可以写为如下的Modelica代码：

```Modelica
within ModelicaByExample.Subsystems.HeatTransfer.Components;
model Rod "Modeling discretized rod"
  import HTC=Modelica.Thermal.HeatTransfer.Components;

  parameter Integer n(start=2,min=2) "Number of rod segments";
  parameter Modelica.SIunits.Temperature T0 "Initial rod temperature";
  Modelica.Thermal.HeatTransfer.Interfaces.HeatPort_a port_a
    "Thermal connector to ambient"
    annotation (Placement(transformation(extent={{-110,-10},{-90,10}})));
  Modelica.Thermal.HeatTransfer.Interfaces.HeatPort_b port_b
    "Thermal connector for rod end 'b'"
    annotation (Placement(transformation(extent={{90,-10},{110,10}})));
  parameter Modelica.SIunits.HeatCapacity C
    "Total heat capacity of element (= cp*m)";
  parameter Modelica.SIunits.ThermalConductance G_wall
    "Thermal conductivity of wall";
  parameter Modelica.SIunits.ThermalConductance G_rod
    "Thermal conductivity of rod";
  Modelica.Thermal.HeatTransfer.Interfaces.HeatPort_a ambient
    "Thermal connector for rod end 'a'"
    annotation (Placement(transformation(extent={{-10,-110},{10,-90}})));
protected
  HTC.HeatCapacitor capacitance[n](each final C=C/n, each T(start=T0, fixed=true))
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        origin={-30,20})));
  HTC.ThermalConductor wall[n](each final G=G_wall/n)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={-30,-20})));
  HTC.ThermalConductor rod_conduction[n-1](each final G=G_rod)
    annotation (Placement(transformation(extent={{-10,-10},{10,10}})));
equation
  for i in 1:n loop
    connect(capacitance[i].port, wall[i].port_b) "Capacitance to walls";
    connect(wall[i].port_a, ambient) "Walls to ambient";
  end for;
  for i in 1:n-1 loop
    connect(capacitance[i].port, rod_conduction[i].port_a)
      "Capacitance to next conduction";
    connect(capacitance[i+1].port, rod_conduction[i].port_b)
      "Capacitance to prev conduction";
  end for;
  connect(capacitance[1].port, port_a) "First capacitance to rod end";
  connect(capacitance[n].port, port_b) "Last capacitance to (other) rod end";
end Rod;
```
此模型有几个有趣的地方可以注意。首先，杆的等分数目由参数n表示：

```Modelica
parameter Integer n(start=2, min=2) "Number of rod segments";
```
参数$n$然后用在下列声明内。参数的目的是指定在杆内热容和热导元素的数目：

```Modelica
HTC.HeatCapacitor capacitance[n](each final C=C/n, each T(start=T0, fixed=true))
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        origin={-30,20})));
  HTC.ThermalConductor wall[n](each final G=G_wall/n)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={-30,-20})));
  HTC.ThermalConductor rod_conduction[n-1](each final G=G_rod)
    annotation (Placement(transformation(extent={{-10,-10},{10,10}})));
```

需要注意的是，如果我们想对组件数组的每个组件进行修改，如加入G=G_rod，我们可以在修改上使用each限定词。我们将在本章节后面修改语句一节里讨论each限定词以及如何让修改应用于元件数组。

现在，我们已经声明了组件数组。然后，我们就可以使用for循环在equation段把热容和热导连接起来：

```Modelica
 for i in 1:n loop
    connect(capacitance[i].port, wall[i].port_b) "Capacitance to walls";
    connect(wall[i].port_a, ambient) "Walls to ambient";
  end for;
  for i in 1:n-1 loop
    connect(capacitance[i].port, rod_conduction[i].port_a)
      "Capacitance to next conduction";
    connect(capacitance[i+1].port, rod_conduction[i].port_b)
      "Capacitance to prev conduction";
  end for;
  ```
  
  我们还需要将杆的端部连接到外部连接器，使该杆可和其他模型相连接：
```Modelica
  connect(capacitance[1].port, port_a) "First capacitance to rod end";
  connect(capacitance[n].port, port_b) "Last capacitance to (other) rod end";
```
这样，我们就能够创建任意次等分的分段杆模型。


## 空间分辨率

现在，我们为Rod模型设定了参数。大家可以看一下分段数量如何影响系统的响应。最终，我们应该看到的是，随着段数增大（或者每段变得更小），解会收敛。

首先，考虑n=3的模型，即：
```Modelica
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model ThreeSegmentRod "Modeling a heat transfer using 3 segment rod subsystem"
  Modelica.Thermal.HeatTransfer.Sources.PrescribedHeatFlow heating
    "Heating actuator"
    annotation (Placement(transformation(extent={{-60,50},{-40,70}})));
  Modelica.Blocks.Sources.Step bc(height=10, startTime=0.5) "Heat profile"
    annotation (Placement(transformation(extent={{-90,50},{-70,70}})));
  Modelica.Thermal.HeatTransfer.Sensors.TemperatureSensor sensor
    annotation (Placement(transformation(extent={{80,-10},{100,10}})));
  Modelica.Thermal.HeatTransfer.Sources.FixedTemperature ambient(T=293.15)
    "Ambient temperature" annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90, origin={0,-80})));
  Components.Rod rod(n=3, C=0.3, G_wall=2.7, T0=293.15, G_rod=1.2)
    annotation (Placement(transformation(extent={{-20,-20},{20,20}})));
equation
  connect(bc.y, heating.Q_flow) annotation (Line(
      points={{-69,60},{-60,60}},
      color={0,0,127}, smooth=Smooth.None));
  connect(heating.port, rod.port_a) annotation (Line(
      points={{-40,60},{-30,60},{-30,0},{-20,0}},
      color={191,0,0},
      smooth=Smooth.None));
  connect(rod.ambient, ambient.port) annotation (Line(
      points={{0,-20},{0,-70},{0,-70}},
      color={191,0,0},
      smooth=Smooth.None));
  connect(rod.port_b, sensor.port) annotation (Line(
      points={{20,0},{80,0}},
      color={191,0,0},
      smooth=Smooth.None));
end ThreeSegmentRod;
```

然后，我们可以从这个模型扩展得到具有更多分段的模型，如：
```Modelica
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model SixSegmentRod "Rod divided into 6 pieces"
  extends ThreeSegmentRod(rod(n=6));
end SixSegmentRod;
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model TenSegmentRod
  extends SixSegmentRod(rod(n=10));
end TenSegmentRod;
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model OneHundredSegmentRod "Rod divided into 100 pieces"
  extends ThreeSegmentRod(rod(n=100));
end OneHundredSegmentRod;
within ModelicaByExample.Subsystems.HeatTransfer.Examples;
model TwoHundredSegmentRod
  extends OneHundredSegmentRod(rod(n=200));
end TwoHundredSegmentRod;
```

如果我们模拟所有这些情况，我们会看到随着n变大，结果似乎收敛到一个共同的解n=10似乎提供了足够精确的解，而不需要引进太多多余的组件：

<img src="./figures8/SegC.png" width=500 align=center>


## 结论

在这一节中，我们已经看到了要如何使用组件数组建立任意大小的组件，并使用for循环将其连接在一起。